# Défi Grosses Data 2018
# Algorithme XGBoost

## Préparation des données

In [13]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import imp
import Annex
imp.reload(Annex)

<module 'Annex' from '/home/marques/Documents/Projet_Meteo/Defi/Annex.py'>

In [16]:
X_train,X_test,Y_train,Y_test,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.2,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='drop')

We are addressing your request.
Data has been imported. Size: (189280, 31)


/home/marques/Documents/Projet_Meteo/Defi/Annex.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='janvier']='1'
/home/marques/Documents/Projet_Meteo/Defi/Annex.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='février']='2'
/home/marques/Documents/Projet_Meteo/Defi/Annex.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='mars']='3'
/home/marques/Documents/Projet_Meteo/Defi/Annex.py:116: SettingWithCopyWarning: 
A value is

Months converted to int.
Dummies added.
Date dropped.
26528 data points deleted. 14.02 %
Train size: 130201, Test size: 32551
Data scaled


In [17]:
X_train.shape

(130201, 38)

## Réseau de neurones avec Keras


In [26]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,SimpleRNN, Lambda,GRU,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D,LSTM,LocallyConnected2D,Convolution2D,Reshape,Conv1D
from keras.utils import np_utils
from keras.optimizers import RMSprop,Nadam,Adam,SGD
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

print(keras.__version__)

2.0.2


#### Définition des séquences du modèle

In [18]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(38, input_dim=38, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [22]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))